Source: https://www.opentopodata.org

In [ ]:
import requests
import shapely
import numpy as np
import pandas as pd

from lets_plot import *
from lets_plot.geo_data import *

In [ ]:
def update_data(data, x, y, lon, lat, elevation, warning=None, print_warning=False):
    if print_warning and warning is not None:
        print("({0}, {1}): {2}".format(lat, lon, warning))
    data["x"].append(x)
    data["y"].append(y)
    data["lon"].append(lon)
    data["lat"].append(lat)
    data["elevation"].append(elevation)
    return data

data = {"x": [], "y": [], "lon": [], "lat": [], "elevation": []}

In [ ]:
PRINT_WARNING = False

country_name = "New Zealand"
file_path = "data/new_zealand.csv"
crop_rect = shapely.Polygon([(150, -48), (150, 0), (180, 0), (180, -48)])

url = lambda lat, lon: "https://api.opentopodata.org/v1/nzdem8m?locations={0},{1}".format(lat, lon)
n, m = 1500, 150

In [ ]:
original_geometry = geocode_countries(country_name).inc_res(4).get_boundaries().iloc[0].geometry
geometry = original_geometry.intersection(crop_rect)
geometry

In [ ]:
bounds = geometry.bounds
bounds

In [ ]:
for x, lon in enumerate(np.linspace(bounds[0], bounds[2], n)):
    for y, lat in enumerate(reversed(np.linspace(bounds[1], bounds[3], m))):
        p = shapely.Point(lon, lat)
        if not geometry.contains(p):
            data = update_data(data, x, y, lon, lat, None, "Out of the area", PRINT_WARNING)
            continue
        response = requests.get(url(lat, lon))
        if not response.ok:
            data = update_data(data, x, y, lon, lat, None, "Response is not OK", PRINT_WARNING)
            continue
        json = response.json()
        if json['status'] != "OK":
            data = update_data(data, x, y, lon, lat, None, "JSON is not OK", PRINT_WARNING)
            continue
        elevation = json["results"][0]["elevation"]
        if elevation is None:
            data = update_data(data, x, y, lon, lat, None, "Elevation is None", PRINT_WARNING)
            continue
        data = update_data(data, x, y, lon, lat, elevation)

df = pd.DataFrame(data)
print(df.shape)
df.head()

In [ ]:
dem_df = df.pivot(index='y', columns='x', values='elevation').fillna(0.0)
print(dem_df.shape)
dem_df.head()

In [ ]:
dem_df.to_csv(file_path, header=False, index=False)